# Hatchet Use Tutorial

Hello welcome to the hatchet tutorial and refrence sheet. Here you can find a quick overview of how to use hatchet and some helpful functions which will aid your use of hatchet. 

## Quick Reference Links

1. [Convenience Functions](#section0)
2. [Loading Data](#section1)
3. [Seeing Data](#section2)
    1. [Interactive Calling Context Tree Quickref](#section2-1)
4. [Data Analysis](#section3)
5. [Filtering and Retreving Queries](#section4)


<a id='section0'></a>

# Part 0 - Convenience Functions

Before we begin we are going to import the hatchet library and define some functions which will help us later. These are not necessary for hatchet . . .

In [ ]:
import hatchet as ht

"""
    The following are convenience functions provided to you for this tutorial, and define some common operations.
    They cannot operate on dataframes produced from eachother,
    so only use them on dataframes directly loaded from a dastaset
"""

def affixColumntoGraphframe(dest_gf, src_gf, colname_dest, colname_src):
    """
        Attaches a column from one graph frame to another. Used by the subsequent functions.
        Params:
            dest_gf: the destination graphframe for the column
            src_gf: the source graphframe for the column
            colname_dest: the target column name on the desination graphframe
            colname_src: the name of the column we would like to transfer from source
    """
    gf_new = dest_gf.copy()
    src_gf = src_gf.copy()
    
    src_gf.dataframe[colname_dest] = src_gf.dataframe[colname_src]
    src_gf.dataframe = src_gf.dataframe.drop(columns=['time (inc)', 'time'])
    
    gf_new.dataframe = gf_new.dataframe \
                        .reset_index() \
                        .join( \
                            src_gf.dataframe.loc[src_gf.dataframe["_missing_node"] == 0].reset_index().set_index(['nid','name']),
                            on=['nid','name'], 
                            lsuffix='_l', 
                            rsuffix='_r'
                        )

    if('_missing_node' in gf_new.dataframe.columns):
        gf_new.dataframe = gf_new.dataframe.drop(columns=['_missing_node'])
    
    removes = [c for c in gf_new.dataframe.columns if '_r' in c]
    renames = {}
    
    for c in gf_new.dataframe.columns:
        if c[-2:] == '_l':
            renames[c] = c[:-2]
    
    gf_new.dataframe = gf_new.dataframe.drop(columns=removes).rename(columns=renames).set_index(['node'])

    gf_new.exc_metrics.append(colname_dest)
    
    return gf_new

def calcSpeedup(gf1, gf2):
    # Calculates the speedup between two graph frames
    # with the same function calls
    # Returns: Copy of GF1 with a new column, speedup
    gf1 = gf1.copy()
    gf2 = gf2.copy()
    gf1.drop_index_levels()
    gf2.drop_index_levels()

    speedup = gf1/gf2
    
    gf_new = affixColumntoGraphframe(gf1, speedup, "speedup", "time")
    
    return gf_new

def calcDiff(gf1, gf2):
    # Calculates the difference in runtimes between two graph frames
    # with the same function calls
    # Returns: Copy of GF1 with a new column, runtimediff
    gf1 = gf1.copy()
    gf2 = gf2.copy()
    gf1.drop_index_levels()
    gf2.drop_index_levels()

    runtimediff = gf1-gf2

    gf_new = affixColumntoGraphframe(gf1, runtimediff, "runtimediff", "time")
    return gf_new
    

def calcImbalance(gf):
    # Calculates the load imbalance across nodes in a single graph frame.
    import numpy as np
    gf1 = gf.copy()
    gf2 = gf.copy()
    gf1.drop_index_levels(function=np.mean)
    gf2.drop_index_levels(function=np.max)
    
    gf1.dataframe['imbalance'] = gf2.dataframe['time'] / gf1.dataframe['time']
    gf1.exc_metrics.append("imbalance")
    
    return gf1

<a id='section1'></a>

# Part 1 - Loading Data

In this tutorial we will show you how to load two different types of files:
1. HPCToolkit Files
2. Caliper Files

## HPCToolkit

In [ ]:
gf_hpc = ht.GraphFrame.from_hpctoolkit('datasets/osu_allgather.1.6.2019-08-26_18-37-57/')

## Caliper

In [ ]:
gf_cali = ht.GraphFrame.from_caliper("datasets/lulesh-scaling/lulesh-annotation-profile-1core.json")

When a data file is passed into a hatchet `from_*` method it creates a GraphFrame. This GraphFrame is a combination Graph and Dataframe. The graph contains the function call hierarchy of in the loaded dataset. The dataframe is indexed by the nodes in our graph and contains all the metrics measured with each function.

<a id='section2'></a>

# Part 2 - Seeing the Data

Once a graphframe has been loaded, we can take a couple of approaches to view our data.

## Dataframe

First, we can look at the data directly through our dataframe. This dataframe can be used like a sortable flat profile.

In [ ]:
gf_cali.dataframe

We can sort this by inclusive time to get a rough idea of our call stack.

In [ ]:
gf_cali.dataframe.sort_values(["time (inc)"], ascending=False)

This can be useful for a quick glance at our data, however, we also have the graph on our graphframe and can visualize that, as well, to get a better idea of the structure of our program hierarchy.

## Console-Based Indented Tree

The following command will print an indented tree showing our data hierarchy and one metric associated with each node.

In [ ]:
print(gf_cali.tree())

This tree can give us a quick picture of our data. It works especially well with small datasets like the one we use here.

Unfortunately, with larger datasets it can be a little difficult to quickly get an idea of the shape of our tree and which nodes might be most important.

We can see this with the hpc_toolkit data.

In [ ]:
print(gf_hpc.tree())

To accomodate for these larger and more complicated datasets we provide an alternative interactive tree visualization designed for use in Jupyter notebooks.

<a id="section2-1"></a>

## Interactive Calling Context Tree

The interactive calling context tree uses an special command called jupyter magics and requires us to load an extension.

In [ ]:
%load_ext hatchet.vis.loader

With this we can now call our visualization on either graph frame with the following magic function. (A magic function is any command that is prefaced by a `%` in jupyter. The calling context tree visualization provides two:
1. `%cct <graphframe>` - This loads the calling context tree visualization
2. `%cct_fetch_query <string>` - This retrieves a query which describes the current visualization configuration and stores it in the passed argument.

When you run the following command it will load the calling context tree visualization. To expand the cell and view the visualization without scrolling within the cell click the box to the immediate left of the visualization.

In [ ]:
%cct gf_hpc

The above tree is still fairly long, however it provides several advantages over a static console based representation.

### Interactive Visualization Features - Layout

The cell above should look very simiar to the image here:

![](layout_annotated.png)

Here you can find descriptions of these key layout elements:

1. This is the `main menu`. It behaves like a dropdown menu in a standard windows-like program. Each of the buttons on this menu will produce a dropdown with additional options. 
    1. `Metrics` contains options related to which metrics are associated with the color of nodes and which are associated with the size. In addition to color, you will see that one metric is labeled "primary," this is the metric to which our `Mass Prune` functionality will be applied.
    2. `Display` contains options related to how we are rendering the colors on the tree as well as which trees we are showing. 
    3. `Query` contains functions related to the reducing the size of our tree and retireving queries which describe the current configuration of the tree for later use.
    
2. Here you will see the `legends`. There are two legends because we encode two metrics on each node to allow for binary analysis. 

3. This is the `tree view`. Here you can scroll, zoom and select nodes on the tree.

### Interactive Visualization Features - Basic Interaction

At a high level, this Calling Context Tree Vis provides most standard functionality expected from an interactive visalization:

1. `Click and Drag` on the tree to pan
2. `Scroll` near the tree to zoom in and out
    1. If you wish to scroll down the page its reccomended you move your mouse to the margins of the jupter notebook.
3. Click the `Display > Reset View` menu option if wish to undo any pan and zoom.
4. Select individual nodes in the tree by `clicking` on them
    1. `Shift + Click` will allow you to select multiple nodes at once.
    2. `Shift + Click` on an already selected node will deselect that node
    
Give each of these commands a try on the prior visualization before continuing.

### Interactive Visualization Features - Advanced Interaction

In the cases of large trees like we have above, it can be frustrating to frequently scroll past many nodes which we may not care about to get to the interesting nodes. Accordingly we provide functionality to prune back this tree to a more manageable size. Here is a list of those more advanced interactions.

1. `Double Click` a node to collapse it and its children into an `aggregrate node`. This hides it from view and shows the average two metrics of all hidden nodes with its color and size.
    1. `Double Click` a collapsed node to re-expand it.
2. `Control + Double Click` a normal node to subsume it into its parent. This can be useful if there are nodes in a call path which add clutter to the tree but are not important to the understanding of how a program functions. For example, wrapper and helper functions that call functions doing real processing.
3. Selecting `Query > Mass Prune` from the dropdown menu will cause a popup will appear. This popup is a tool you can use to rapidly cut away uninteresting nodes in your program:

![](popup_layout.png)

Layout descriptions:
1. The top bar of the popup can be grabbed to drag and drop this window anywhere in the visualization.
2. The violet top bars in this histogram show the frequency of nodes which can be pruned away within a certain metric range. The metric range they are grouped by is the `primary` "color" metric.
3. The bottom bars show the number of nodes in this metric range which are zero-value internal nodes which are the ancestors of these nodes. They will not be pruned away until a full subtree of nodes is removed.
4. These sliders are your primary interaction mechanism in this tool. `Click + Drag` either slider to prune away nodes outisde of the threshold specified under each slider.

Here is the HPCToolkit data once again so you can try out this advanced functionality without having to scroll all the way back up.

In [ ]:
%cct gf_hpc

<a id='section3'></a>

# Part 3 - Data Analysis

Now that we have out tools for viewing and loading data, we will demonstrate some mechanisms for interacting with and modifying our graphframe.

### Mathemetical Operators

If we have two graph frames of similar shapes we can perform operations across our whole graphframe which applies to each node. A graphframe supports `+`,`-`,`*`,`/` between two graph frames.

In the following example we will use the divide operator to calculate speedup between two graphframes. To do this, first we must load a second lulesh graphframe.

In [ ]:
gf_lulesh_64 = ht.GraphFrame.from_caliper("datasets/lulesh-scaling/lulesh-annotation-profile-64cores.json")

In [ ]:
gf_lulesh_512 = ht.GraphFrame.from_caliper("datasets/lulesh-scaling/lulesh-annotation-profile-512cores.json")

In [ ]:
gf_lulesh_speedup = gf_lulesh_64 / gf_lulesh_512

The result of this division, speedup, is stored on each node in gf_lulesh_speedup as "time".

In [ ]:
gf_lulesh_speedup.dataframe

We could visualize the results of this calculation like any other tree. However if we wish to see time and speedup at the same time on a single node, to identify clear optimization targets, this can be difficult with the console based tree. We need to print out both trees and compare them side by side.

If we combine our speedup graphframe with our original graphframe through the use of the following helper function, we can visualize speedup and time on the each node at the same time. We can also view other data like load imbalance.

In [ ]:
gf_lulesh_speedup = calcSpeedup(gf_lulesh_64, gf_lulesh_512)

In [ ]:
gf_lulesh_speedup = calcImbalance(gf_lulesh_speedup)

In [ ]:
%cct gf_lulesh_speedup

On this graph you will see that the `speedup` of a node is indicated with the blue-red color gradient, while `time` is shown with the node's relative size; bigger nodes mean more time. If you select `display > color map > inverted` from the dropdown menu, lower numbers will be highlighted red, indicating poor speedup.

With this configuration we can see that the bottom left leaf and lower right trees look the most problematic, with relatively low speedup as we scale from 64 to 256 cores. To simplify our tree and focus only on this subtree we can manually collapse the top two subtrees at level 3 by double clicking the internal nodes we want them to `collapse into`. The result should be a tree that looks like the following:

![](manual_prune.png)

We can also remove some of these mpi nodes which seem to be cluttering up our tree by `ctrl + double click` on each you wish to remove.

<a id='section4'></a>

# Part 4 - Filtering and Retireving Queries from the Visualization

Above, we showed an example of modifying our tree via the interactive visualization: removing nodes which we are not useful to our analysis and clutter a tree. Unfortunately, these changes will be lost if the visualization is reloaded; so Hatchet provides functionality which allows us to reduce our source graphframe. The `filter` function.

## Filter

The `filter` class method can be called on a graphframe like so:

In [ ]:
gf_lulesh_low_speedup = gf_lulesh_speedup.filter(lambda x: x.speedup < 1);
gf_lulesh_high_speedup = gf_lulesh_speedup.filter("MATCH (a) WHERE a.'speedup' >= 1");

This filter function accepts two types of queries:
1. lambda expressions that can operate on any metric on your dataframe, like the above example
2. string based queries using a hatchet-specific query language. 

We will not go into detail explaining the syntax of the query language here; however it is much more powerful than the lambda expressions and allows the creation of more complex queries which are hard to specify mathematically. 

## Retriving a text based query from the visualization

In the last part of this tutorial we modified a tree using manual interactions but no strict pattern of inclusivity: we removed MPI nodes and collapsed subtrees we weren't intrested in. If we want to apply these changes by filtering our graphframe we would need to articulate these series of decisions in a complex lambda function or string-based query. 

To avoid this manual transfer, the interactive visualization provides a means to export a snapshot of the tree in query-language form back to the jupyter notebook, which can be applied to the original graphframe and stored for later use.

**To export the query, return to the visualization and select `Query > Get Snapshot Query` from top menu.**

After that you can run the following command to store the query in the argument to our magic function: `nodeSubselection`.


In [ ]:
%cct_fetch_query nodeSubselection

In [ ]:
nodeSubselection

By applying this query to our original graphframe, it modfies our data on the python side so we can now call our visualizations or examine our dataframes on a more narrow potion of our calling context tree.

In [ ]:
drop_index_levels()
gf_snapshot = gf_lulesh_speedup.filter(nodeSubselection)

In [ ]:
%cct gf_snapshot

We can also store this query for later use.

In [ ]:
with open('lulesh_optimization_target_query.txt', 'w') as f:
    f.write(nodeSubselection)

And, when we want to re-use it, we can read it into a variable and reduce our graphframe pre-visualization.

In [ ]:
query = ''
with open('lulesh_optimization_target_query.txt', 'r') as f:
    query = f.read()

# If we closed this notebook an re-oepned it, 
# we would need to re-caluclate speedup, because 
# it is a derived metric which was not stored on the source graphframes
gf_lulesh_64 = ht.GraphFrame.from_caliper("datasets/lulesh-scaling/lulesh-annotation-profile-64cores.json")
gf_lulesh_512 = ht.GraphFrame.from_caliper("datasets/lulesh-scaling/lulesh-annotation-profile-512cores.json")
gf_lulesh_speedup = calcSpeedup(gf_lulesh_64, gf_lulesh_512)

print(query)

recovered_gf_lulesh_snapshot = gf_lulesh_speedup.filter(query)

In [ ]:
%cct recovered_gf_lulesh_snapshot